In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.


# Goal of this Notebook is to solve task 3:
**What do we know about virus genetics, origin, and evolution? What do we know about the virus origin and management measures at the human-animal interface?
**

With the help of Natural language processing we are aiming to help our doctors and researchers. Our hope is to providea faster and better access to the information that has been piublished by other researchers. This access should be able to provide valuable insights from a huge database.

to be able to provide valuable insights, we have interviewd  Prof. Dr. Hassan Vahidnezhad ,Faculty professor of Jefferson Institute of Molecular Medicine and the Department of Dermatology and Cutaneous Biology, Thomas Jefferson University Hospital, Philadelphia, PA.
he helped us to understand how we have to define our objectives to reach the goal of task 3. he has defined 4 subcategories that could fit into mentioned task.

1. Characteristics
2. Virulence
3. Origin
4. Role of virus genetics in Infection and treatment

in order to be able to gain insights from text with computer vectorization with NLP comes into play. however it is a matter of great importance to find out which vectorization is understanding better the meaning of text and their priorization. since the database available for us is without any label, therefore we need to follow unsupervised learning to achieve our goal. one way to realize if our methodology and chosen path is correct is through visualization.
flexibility of scatter plots for use in exploratory analysis has been proven, thus scatterplots are a very effective way of presentations and gaining valuable insights regarding data. however as data grows in scale and complexity some editions to the traditional scatter plots needs to be implemented.In order to achieve the goal of visualization it is a matter of great importance to identify the factors that influence the appropriateness of desired scatterplot. these factors should be defined based on specific data characterisitics and tasks.Thereafter valuable insights could be presented as a table or histogram in final steps.


# ***Our approach in steps ***


1. reading the data
2. cleaning the data (making the data more managable, keeping the documents that are helpful to us, removing noise and outliers with respect to this specific task)
3. preprocessing the texts effectively
4. with the help of NLP and method .... vectorized the body text of each article
5. reducing the dimension of each document vector to 2 with the help pf PCA method (Principal Component Analysis)
6. with the help elbow method finding the optimal number of clustering
7. make the optimal clustering of the similar research papers
    (visualized in 2 dimension plot)
8. Investigate the clusters visually on the plot
9. calculate the accuracy and f1 score of clusters with the help of KNN method
10. 
11. 
12. 
13. 




# ***Loading the data**
** this part of our notebook is inspired by a notebook from MaksimEkin in kaggle**

cite: @inproceedings{COVID-19 Literature Clustering,
    author = {Eren, E. Maksim. Solovyev, Nick. Nicholas, Charles. Raff, Edward},
    title = {COVID-19 Literature Clustering},
    year = {2020},
    month = {April},
    location = {University of Maryland Baltimore County (UMBC), Baltimore, MD, USA},
    note={Malware Research Group},
    url = {\url{https://github.com/MaksimEkin/COVID19-Literature-Clustering}},
    howpublished = {TBA}
} 

1. We are going to extract 'full body texts' , 'paper id', and 'abstract'  from jason files. 
2. We load the meta data too, because some of their characteristics like author, sha and journal name will be helpful for us.
3. merging them all into a dataframe together

* libraries that have been used in this part: 
* * panda
* * numpy
* * json

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import json

import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [ ]:
###Loading meta data###

root_path = '/kaggle/input/CORD-19-research-challenge/'
metadata_path = f'{root_path}metadata.csv'
meta_df = pd.read_csv(metadata_path, dtype={
    'pubmed_id': str,
    'Microsoft Academic Paper ID': str, 
    'doi': str
})
meta_df.head()

In [ ]:
### Fetch All of JSON File Path###


all_json = glob.glob(f'{root_path}/**/*.json', recursive=True)
len(all_json)

In [ ]:
### helper function ###

#file reader class
class FileReader:
    def __init__(self, file_path):
        with open(file_path) as file:
            content = json.load(file)
            try:
                self.paper_id = content['paper_id']
            except Exception as e:
                self.paper_id = ''
            self.abstract = []
            self.body_text= []
            
            # Abstract
            try:
                for entry in content['abstract']:
                    self.abstract.append(entry['text'])
            except Exception as e:
                pass
            # Body text
            
            try:
                for entry in content['body_text']:
                    self.body_text.append(entry['text'])
            except Exception as e:
                pass
            
            self.abstract = '\n'.join(self.abstract)
            self.body_text = '\n'.join(self.body_text)
    def __repr__(self):
        return f'{self.paper_id}:{self.abstract[:200]}... {self.body_text[:200]}...'
    
first_row = FileReader(all_json[0])
print(first_row)

In [ ]:
##Load the Data into DataFrame##

#Using the helper functions, let's read in the articles into a DataFrame that can be used easily:
    
dict_ = {'paper_id': [], 'abstract': [], 'body_text': [], 'authors': [], 'title': [], 'journal': [] }
for idx, entry in enumerate(all_json):
    if idx % (len(all_json) // 100) == 0:
        print(f'Processing index: {idx} of {len(all_json)}')
    content = FileReader(entry)
    
    # get metadata information
    meta_data = meta_df.loc[meta_df['sha'] == content.paper_id]
    # no metadata, skip this paper
    if len(meta_data) == 0:
        continue
    
    dict_['paper_id'].append(content.paper_id)
    dict_['abstract'].append(content.abstract)
    dict_['body_text'].append(content.body_text)
    
        
    # get metadata information
    meta_data = meta_df.loc[meta_df['sha'] == content.paper_id]
    dict_['title'].append(meta_data['title'].values[0])
    dict_['authors'].append(meta_data['authors'].values[0])

    
    # add the journal information
    dict_['journal'].append(meta_data['journal'].values[0])
    
df_covid = pd.DataFrame(dict_, columns=['paper_id','abstract', 'body_text', 'authors', 'title', 'journal'])
df_covid.head()



# Preprocessing 
after creating a dataframe we try to do some cleaning and feature engineering on our data
1. removing duplicates (articles that are existing more than once get emitted)
2. adding extra column to our data frame which tages the language of the papers
3. keeping the articles that are in english language
4. based on our user interview (Dr...) adding extra column to our data which labels the articles based on specifications of task 3
5. keeping the articles that should be definitley considered for task 3 thus reducing the noise
6. preparing the text to be ready for vectorization in the next steps

In [ ]:
##droping the duplicates##
dict_ = None

df_covid.info()

df_covid.drop_duplicates(['abstract', 'body_text'], inplace=True)

df_covid['abstract'].describe(include='all')
df_covid['body_text'].describe(include='all')


## droping the null values##
# drop Null vales:
df_covid.dropna(inplace=True)

In [ ]:
import sys
!{sys.executable} -m pip install --upgrade pip

In [ ]:
!pip install langdetect


In [ ]:
## labelling the languages of articles in a separate column##

from langdetect import detect
df_covid['language'] = df_covid['title'].apply(detect)

#checking what languages are present in the dataset
uniqueValues = df_covid['language'].unique()

df_covid.language.value_counts()


In [ ]:
import seaborn as sns
plt.figure(figsize=(10,5))
sns.set_style('darkgrid')
sns.countplot(x='language',data=df_covid,palette='viridis')
plt.title('language distribution')

**Taking a look at the distribution of languages in available articles in this data set, 
it is logical to keep only the papers with english language only **

In [ ]:
## keeping english language only ##

df_covid = df_covid.loc[df_covid['language'] == 'en']
df_covid.info()

*based on our user interview (Dr...) adding extra column to our data which labels the articles based on specifications of task 3*
we concluded that we should label the articles based on specific keywords that are related to **'Covid 19'** , **'SARS'**, **'MERS', 'Proteins'** and **'Genomes'**

In [ ]:
#finding papers takling about covid and sars and mers, proteins, genome


# a function that will work as a searching machine to find specific keywords in papers
import re
def pattern_searcher(search_str:str, search_list:str):

    search_obj = re.search(search_list, search_str)
    if search_obj :
        return True
    else:
        return False
    
###keywords related to covid that we are looking for in literatures  
covid_list = ['covid',
                    'coronavirus disease 19',
                    'sars cov 2', # Note that search function replaces '-' with ' '
                    '2019 ncov',
                    '2019ncov',
                    r'2019 n cov\b',
                    r'2019n cov\b',
                    'ncov 2019',
                    r'\bn cov 2019',
                    'novel corona',
                    'coronavirus 2019',
                    'wuhan pneumonia',
                    'wuhan virus',
                    'wuhan coronavirus',
                    r'coronavirus 2\b']
pattern_covid = '|'.join(covid_list)

#adding extra column that tags papers that are related to covid19
df_covid['covid_match'] = df_covid['body_text'].apply(lambda x: pattern_searcher(search_str=x, search_list=pattern_covid))



###keywords related to SARS that we are looking for in literatures
sars_list = [r'\bsars\b','severe acute respiratory syndrome']
pattern_sars = '|'.join(sars_list)

#adding extra column that tags papers that are related to SARS
df_covid['sars_match'] = df_covid['body_text'].apply(lambda x: pattern_searcher(search_str=x, search_list=pattern_sars))




###keywords related to MERS that we are looking for in literatures
mers_list = [r'\bmers\b','middle east respiratory syndrome']
pattern_mers = '|'.join(mers_list)

#adding extra column that tags papers that are related to MERS
df_covid['mers_match'] = df_covid['body_text'].apply(lambda x: pattern_searcher(search_str=x, search_list=pattern_mers))



###keywords related to protein and receptor surface protein that we are looking for in literatures
protein_list = ['protein','surface protein', 'proteome', 'proteomics', 'proteomic','proteins']
pattern_protein = '|'.join(protein_list)

#adding extra column that tags papers that are related to Protein
df_covid['protein_match'] = df_covid['body_text'].apply(lambda x: pattern_searcher(search_str=x, search_list=pattern_protein))



###keywords related to genomic sequences that we are looking for in literatures
genom_list=['genomic','genomics', 'genome', 'genome sequence', 'genome-wide']
pattern_genom = '|'.join(genom_list)

#adding extra column that tags papers that are related to Genomic sequences
df_covid['genom_match'] = df_covid['body_text'].apply(lambda x: pattern_searcher(search_str=x, search_list=pattern_genom))



In [ ]:
df_covid.head()

In [ ]:
### only keeping the papers that should be most related to task3 ###
#

**Only keeping the papers that should be the most related to task 3**
therefore we decided to keep the papers that have one of these conditions:
1. talking abour *covid* **and** *protein* **and** *genomes*
2. talking abour *MERS* **and** *protein* **and** *genomes*
3. talking abour *SARS* **and** *protein* **and** *genomes*


In [ ]:
### only keeping the papers that should be most related to task3 ###
df_covid_task3 = df_covid.loc[((df_covid['covid_match'] == True) & (df_covid['protein_match']== True) & (df_covid['genom_match']== True)) | ((df_covid['mers_match']== True) & (df_covid['protein_match']== True) & (df_covid['genom_match']== True)) | ((df_covid['sars_match']== True) & (df_covid['protein_match']== True) & (df_covid['genom_match']== True))]
df_covid_task3.info()

In [ ]:
df_covid_task3.head()

**preparing the text to be ready for vectorization in the next steps**

now we need to convert our data into something that a computer can understand.
1. removing useless data which we call them stopwords
2. finding the words with the same meaning with the help of wordnet database
3. cleaning the string



In [ ]:
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords

from nltk.stem.wordnet import WordNetLemmatizer

lemma = WordNetLemmatizer()


stopword_set = set(stopwords.words('english')+['a','at','s','for','was', 'we', 'were', 'what', 'when', 'which', 'while', 'with', 'within', 'without', 'would', 'seem', 'seen','several', 'should','show', 'showed', 'shown', 'shows', 'significantly', 'since', 'so', 'some', 'such','obtained', 'of', 'often', 'on', 'our', 'overall','made', 'mainly', 'make', 'may', 'mg','might', 'ml', 'mm', 'most', 'mostly', 'must', 'each', 'either', 'enough', 'especially', 'etc','had', 'has', 'have', 'having', 'here', 'how', 'however', 'upon', 'use', 'used', 'using', 'perhaps', 'pmid','can', 'could', 'did', 'do', 'does', 'done', 'due', 'during'])

stop_set=['a','at','s','for','was', 'we', 'were', 'what', 'when', 'which', 'while', 'with', 'within', 'without', 'would', 'seem', 'seen','several', 'should','show', 'showed', 'shown', 'shows', 'significantly', 'since', 'so', 'some', 'such','obtained', 'of', 'often', 'on', 'our', 'overall','made', 'mainly', 'make', 'may', 'mg','might', 'ml', 'mm', 'most', 'mostly', 'must', 'each', 'either', 'enough', 'especially', 'etc','had', 'has', 'have', 'having', 'here', 'how', 'however', 'upon', 'use', 'used', 'using', 'perhaps', 'pmid','can', 'could', 'did', 'do', 'does', 'done', 'due', 'during']

#definig a function that cleans the string of full body text
def process(string):
    string=' '+string+' '
    string=' '.join([word if word not in stopword_set else '' for word in string.split()])
    string=re.sub('\@\w*',' ',string)
    string=re.sub('\.',' ',string)
    string=re.sub("[,#'-\(\):$;\?%]",' ',string)
    string=re.sub("\d",' ',string)
    string=re.sub(r'[^\x00-\x7F]+',' ', string)
    for i in stop_set:
        string = re.sub(' ' +i+' ', ' ', string)
    string=" ".join(lemma.lemmatize(word) for word in string.split())
    string=re.sub('( [\w]{1,2} )',' ', string)
    string=re.sub("\s+",' ',string)
    string=string.replace('[', '')
    string=string.replace(']', '')
    return string.split()